In [9]:
import cellCnn
import importlib
importlib.reload(cellCnn)
import random
import numpy as np
### (from: https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/NK_cell_ungated.ipynb ) AND https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/PBMC.ipynb
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt
from cellCnn.ms.utils.helpers import get_chunks
from cellCnn.ms.utils.helpers import print_regression_model_stats
from cellCnn.plotting import plot_results
from cellCnn.utils import mkdir_p
from cellCnn.utils import save_results
from cellCnn.ms.utils.helpers import get_fitted_model
from cellCnn.ms.utils.helpers import split_test_valid_train
from cellCnn.ms.utils.helpers import calc_frequencies, get_chunks_from_df
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


In [10]:
#%reset

In [11]:
##### state vars
cytokines = ['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 'CD25', 'CD27', 'CD28', 'CD3',
             'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10',
             'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3', 'IL.4', 'IL.6', 'IL.9', 'TNF.a']
infile = 'cohort_denoised_clustered_diagnosis_patients.csv'
indir = 'data/input.nosync'
outdir = 'out_ms_default'
rand_seed = 123
train_perc = 0.7
test_perc = 0.3
batch_size_pheno = 840 # deprecated  # so a size of 8425 is about equally sized in batches
batch_size_cd4 = 550  # deprecated #so a size of 550 gets me 16 batches for cd4
## information from ms_data project
cluster_to_celltype_dict = {0: 'b', 1: 'cd4', 3: 'nkt', 8: 'cd8', 10: 'nk', 11: 'my', 16: 'dg'}

In [12]:
np.random.seed(rand_seed)
mkdir_p(outdir)
df = pd.read_csv(os.path.join(indir, infile), index_col=0)
df = df.drop_duplicates()  ### reduces overfitting at cost of fewer data
df.shape
##### no duplicates in

(16889, 38)

In [13]:
plt.figure()
patients_clusters_conf_table = pd.crosstab(df['gate_source'], df['cluster'])
sns.heatmap(patients_clusters_conf_table, annot=False, vmax=100)
plt.show()
#plt.savefig('images/patient_vs_cluster_conf_table.png')
patients_clusters_conf_table

/var/folders/p1/5d38cf1d6tvdyny38zb_66540000gn/T/ipykernel_46232/3510062378.py:4: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


cluster,0,1,3,8,10,11,16
gate_source,,,,,,,
3.0,60,119,6,30,5,5,0
4.0,15,46,0,45,2,1,13
5.0,33,131,0,70,2,2,4
8.0,24,122,0,31,10,11,4
9.0,72,270,2,137,11,10,4
...,...,...,...,...,...,...,...
128.0,4,6,4,6,1,6,0
130.0,24,82,2,56,8,4,0
131.0,20,48,2,14,13,7,0


In [71]:
print('Mean abundancies')
print('b: ' + str(patients_clusters_conf_table.iloc[:,0].mean()))
print('CD4: ' + str(patients_clusters_conf_table.iloc[:,1].mean()))
print('NKT: ' + str(patients_clusters_conf_table.iloc[:,2].mean()))
print('CD8: ' + str(patients_clusters_conf_table.iloc[:,3].mean()))
print('NK: ' + str(patients_clusters_conf_table.iloc[:,4].mean()))
print('My: ' + str(patients_clusters_conf_table.iloc[:,5].mean()))
print('dg: ' + str(patients_clusters_conf_table.iloc[:,6].mean()))


Mean abundancies
b: 36.37096774193548
CD4: 141.98387096774192
NKT: 5.903225806451613
CD8: 64.04838709677419
NK: 12.580645161290322
My: 6.741935483870968
dg: 4.774193548387097


In [72]:
# pitch: key = gate_source, value = dict{diagnosis: df OR freq?}
rrms_df = df[df['diagnosis'] == 'RRMS']
rrms_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                    rrms_df.groupby('gate_source')}
nindc_df = df[df['diagnosis'] == 'NINDC']
nindc_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                     nindc_df.groupby('gate_source')}

KeyboardInterrupt: 

In [15]:
print('RRMS cell-type abundances')
rrms_df.groupby('cluster').count()
print('Mean abundancy / patient is 273,032258065')

RRMS cell-type abundances


,CD69,CXCR5,CCR2,CD57,CD28,IL.9,IL.4,CD27,IL.3,IL.13,...,CD25,CD4,CD3,IL.21,IL.17A,CXCR4,CCR4,CD14,diagnosis,gate_source
cluster,,,,,,,,,,,,,,,,,,,,,
0,1114,1114,1114,1114,1114,1114,1114,1114,1114,1114,...,1114,1114,1114,1114,1114,1114,1114,1114,1114,1114
1,4302,4302,4302,4302,4302,4302,4302,4302,4302,4302,...,4302,4302,4302,4302,4302,4302,4302,4302,4302,4302
3,147,147,147,147,147,147,147,147,147,147,...,147,147,147,147,147,147,147,147,147,147
8,2113,2113,2113,2113,2113,2113,2113,2113,2113,2113,...,2113,2113,2113,2113,2113,2113,2113,2113,2113,2113
10,401,401,401,401,401,401,401,401,401,401,...,401,401,401,401,401,401,401,401,401,401
11,216,216,216,216,216,216,216,216,216,216,...,216,216,216,216,216,216,216,216,216,216
16,171,171,171,171,171,171,171,171,171,171,...,171,171,171,171,171,171,171,171,171,171


In [16]:
print('NINDC cell-type abundances')
nindc_df.groupby('cluster').count()

NINDC cell-type abundances


,CD69,CXCR5,CCR2,CD57,CD28,IL.9,IL.4,CD27,IL.3,IL.13,...,CD25,CD4,CD3,IL.21,IL.17A,CXCR4,CCR4,CD14,diagnosis,gate_source
cluster,,,,,,,,,,,,,,,,,,,,,
0,1141,1141,1141,1141,1141,1141,1141,1141,1141,1141,...,1141,1141,1141,1141,1141,1141,1141,1141,1141,1141
1,4501,4501,4501,4501,4501,4501,4501,4501,4501,4501,...,4501,4501,4501,4501,4501,4501,4501,4501,4501,4501
3,219,219,219,219,219,219,219,219,219,219,...,219,219,219,219,219,219,219,219,219,219
8,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,...,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858
10,379,379,379,379,379,379,379,379,379,379,...,379,379,379,379,379,379,379,379,379,379
11,202,202,202,202,202,202,202,202,202,202,...,202,202,202,202,202,202,202,202,202,202
16,125,125,125,125,125,125,125,125,125,125,...,125,125,125,125,125,125,125,125,125,125


In [17]:
importlib.reload(cellCnn.ms.utils.helpers)
from cellCnn.ms.utils.helpers import *

#### here we could see freq differences across the 2 groups
print('Frequencies: ')
rrms_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df in
                      rrms_patients2df.items()}
nindc_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df
                       in nindc_patients2df.items()}
print('DONE')
# we got 31 patients each

Frequencies: 
DONE


In [18]:
#### To get true frequencies we need to get rid of 0 entries (there are patient without some cells type due to data)
print('RRMS')
rrms_freq_df = pd.DataFrame(list(rrms_patients_freq.values()), columns=list(cluster_to_celltype_dict.values()))
rrms_freq_df = rrms_freq_df.replace(0, np.NaN)  ## this lets us skip the 0 entries
rrms_freq_df.describe()

RRMS


,b,cd4,nkt,cd8,nk,my,dg
count,31.000000,31.000000,28.000000,31.000000,31.000000,31.000000,25.000000
mean,0.145822,0.470019,0.034702,0.242621,0.047500,0.038641,0.029826
std,0.062796,0.127876,0.043881,0.068406,0.031251,0.041190,0.026293
min,0.044118,0.222222,0.003953,0.120543,0.008264,0.004662,0.001832
25%,0.098750,0.393336,0.008248,0.193943,0.022284,0.017099,0.011655
50%,0.136364,0.461538,0.018236,0.243386,0.045455,0.023810,0.019802
75%,0.190358,0.543535,0.032926,0.283517,0.057529,0.044735,0.042857
max,0.285714,0.738540,0.178571,0.349057,0.125000,0.222222,0.128205


In [19]:
print('NINDC')
nindc_freq_df = pd.DataFrame(list(nindc_patients_freq.values()), columns=list(cluster_to_celltype_dict.values()))
nindc_freq_df = nindc_freq_df.replace(0, np.NaN)
nindc_freq_df.describe()

NINDC


,b,cd4,nkt,cd8,nk,my,dg
count,31.000000,31.000000,30.000000,31.000000,30.000000,29.000000,26.000000
mean,0.160800,0.440072,0.043986,0.245588,0.059403,0.039039,0.020230
std,0.107951,0.155058,0.038204,0.094339,0.047733,0.033544,0.020225
min,0.033149,0.189189,0.001524,0.046559,0.011236,0.001706,0.002841
25%,0.076150,0.329932,0.012810,0.206028,0.024907,0.014587,0.007189
50%,0.142857,0.390476,0.039086,0.247573,0.037789,0.031250,0.017101
75%,0.215665,0.516367,0.067342,0.290557,0.086959,0.050000,0.024923
max,0.433198,0.784572,0.142857,0.410061,0.200000,0.142857,0.106557


In [75]:
##### CLASSIFICATION TASK DEEASE state

importlib.reload(cellCnn.ms.utils.helpers)
from cellCnn.ms.utils.helpers import *


mean_abundace = 273 # for desease states
batch_sizes = [40, 80, int(mean_abundace*0.6),int(mean_abundace*0.8),int(mean_abundace*0.9),mean_abundace, int(mean_abundace*1.25)]
batch_size_dict_class = dict()
#batch_size = 20
cluster = None
for batch_size in batch_sizes:
    ### desease states 1 = RRMS and 0 = NINDC
    selection_pool_rrms_class, too_few_data_rrms_class = get_chunks_from_df(rrms_patients2df,
                           freq_df=rrms_patients_freq,
                           desease_state=1,
                           cluster=cluster,
                           batch_size=batch_size)
    selection_pool_nindc_class, too_few_data_nindc_class = get_chunks_from_df(nindc_patients2df,
                           freq_df=nindc_patients_freq,
                           desease_state=0,
                           cluster=cluster,
                           batch_size=batch_size)
    #todo make sure list are equally long:
    if len(selection_pool_rrms_class) > len(selection_pool_nindc_class):
        selection_pool_rrms_class = selection_pool_rrms_class[:len(selection_pool_nindc_class)]
    elif len(selection_pool_rrms_class) < len(selection_pool_nindc_class):
        selection_pool_nindc_class = selection_pool_nindc_class[:len(selection_pool_rrms_class)]

    all_chunks = selection_pool_rrms_class + selection_pool_nindc_class
    np.random.shuffle(all_chunks)

    X = [selection[0] for selection in all_chunks]
    freqs = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]

    batch_size_dict_class[batch_size] = (X, freqs, Y)

print('prep done')

prep done


In [79]:
importlib.reload(cellCnn.ms.utils.helpers)
importlib.reload(cellCnn.model)
from cellCnn.ms.utils.helpers import *
from cellCnn.model import CellCnn

splits = 3
model_container_class = {}
for batch_size, values in batch_size_dict_class.items():
    kf = StratifiedKFold(n_splits=splits, random_state=rand_seed, shuffle=True)
    X, Y = values[0], values[2]
    X_test, X_train, X_valid, y_test, y_train, y_valid = split_test_valid_train(X=X,
        y=Y,
        test_perc=test_perc,
        train_perc=train_perc,
        valid_perc=0.5, seed=rand_seed)
    X = np.array([*X_train, *X_valid])
    y = np.array([*y_train, *y_valid])
    if len(X[0]) < splits:
        continue

    i = 1
    for train_idx, valid_idx, in kf.split(X=X, y=y):
        outdir_pheno = f'ms_pheno_class_{batch_size}_{i}'
        i = i+1
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]
        model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                                 nsubset=int(batch_size*len(X_train)/2),
                                 nfilters=[3, 15, 25, 35], coeff_l1=0,
                                 max_epochs=100, nrun=30, learning_rate=None,
                                 ncell=batch_size,
                                 per_sample=False, regression=False,
                                 outdir=outdir_pheno, verbose=False)
        model_container_class[batch_size] = model

107/107 [==============================] - 0s 2ms/step - loss: 0.6939 - accuracy: 0.5000


In [84]:
accs = []
i = 0
stats_dict_class = dict()
old_batch_size = None
for batch_size, model in model_container_class.items():
    results = model.results
    test_pred = model.predict(X_test)
    train_pred = model.predict(X_train)
    valid_pred = model.predict(X_valid)

    test_pred_abs = [1 if pred[0]>pred[1] else 0 for pred in test_pred ]
    train_pred_abs = [1 if pred[0]>pred[1] else 0 for pred in train_pred ]
    valid_pred_abs = [1 if pred[0]>pred[1] else 0 for pred in valid_pred ]
    test_acc = accuracy_score(y_test, test_pred_abs)
    train_acc = accuracy_score(y_train, train_pred_abs)
    valid_acc = accuracy_score(y_valid, valid_pred_abs)
    print(f'Test Acc : {test_acc}')
    print(f'Train Acc : {train_acc}')
    print(f'Valid Acc : {valid_acc}')
    accs.append(test_acc)

    if i == 2:
        mean_test_acc = sum(accs)/len(accs)
        accs = []
        i = 0
        print(f'BS: {batch_size}')
        print(f'Mean Acc: {str(mean_test_acc)}')
        stats_dict_class[batch_size] = {'mean_acc': mean_test_acc}
    i = i +1

print('DONE')

/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Test Acc : 0.0
Train Acc : 0.3
Valid Acc : 0.4


/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Test Acc : 0.0
Train Acc : 0.5
Valid Acc : 0.4


/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


...

Test Acc : 0.3333333333333333
Train Acc : 0.3
Valid Acc : 0.2
BS: 245
Mean Acc: 0.16666666666666666


/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Test Acc : 0.3333333333333333
Train Acc : 0.1
Valid Acc : 0.4


/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Test Acc : 0.3333333333333333
Train Acc : 0.0
Valid Acc : 0.4
BS: 341
Mean Acc: 0.3333333333333333
DONE


In [82]:
#### NOTE:
#### it performs Strat.KFold anyways (even if i dont manually divide them into train valids... )
#### bad thing is that i dont have any test yet
#### made Errors:
#### 1. I put in y_unique as the true class labels (no integer in a range) and got a list indes out of bound
print('######################################')
print('######################################')
print(' CLASSIFICATION task DONE')
print('######################################')
print('######################################')


######################################
######################################
 CLASSIFICATION task DONE
######################################
######################################


In [131]:
# plot the results of the CellCnn analysis for the test samples in the output directory
_ = plot_results(model.results, X_test, y_test,
                 cytokines, outdir + '/plots', filter_response_thres=0,
                 filter_diff_thres=0.2, group_a='RRMS', group_b='NINDC')
print('done')
### plotting stuff


### RESULTS: Also es ist stark abhängig vom Zufall aber auch von der batchsize.
### nsubsets und ncell ist mir noch ein Rätsel


# This is classification. ..
# calculate area under the ROC curve for the test set

done


In [ ]:
#####################################
#################### REGRESSION desease state
#####################################

In [85]:
print('######################################')
print('######################################')
print('########### REGRESSION desease state ###########')
print('######################################')
print('######################################')

model_container_reg = []
X=batch_size_dict_class[80][0]
Y=batch_size_dict_class[80][2]
X_test, X_train, X_valid, y_test, y_train, y_valid = split_test_valid_train(X=X,
    y=Y,
    test_perc=test_perc,
    train_perc=train_perc,
    valid_perc=0.5, seed=rand_seed)
X = np.array([*X_train, *X_valid])
y = np.array([*y_train, *y_valid])
kf = StratifiedKFold(n_splits=3)
count=1 # this is just there to save the models of all iterations
batch_size = 200
for train_idx, valid_idx, in kf.split(X=X, y=y):
    outdir = f'ms_pheno_reg_{batch_size}_{count}'
    count = count +1
    X_train, X_valid = X[train_idx], X[valid_idx]
    y_train, y_valid = y[train_idx], y[valid_idx]
    model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                             nsubset=int(batch_size*len(X_train)/2),
                             nfilters=[3, 15, 25, 35], coeff_l1=0,
                             max_epochs=100, nrun=30, learning_rate=None,
                             ncell=batch_size,
                             per_sample=True, regression=True,
                             outdir=outdir_pheno, verbose=False)
    model_container_reg.append(model)

######################################
######################################
########### REGRESSION desease state ###########
######################################
######################################


In [39]:
mses = []
for model in model_container:
    print('BS:' + str(batch_size))
    results = model.results
    test_pred = model.predict(X_test)
    mses.append(print_regression_model_stats(test_pred, y_test)[0])

print(f'Mean MSE: {str(sum(mses)/len(mses))}')

# 100 Mean MSE: 0.31166256644638163
# 200 0.36675149071264573

BS:200

Model predictions:
 [0.42304914196332294, 0.5106603304545084, 0.36440641681353253]

True phenotypes:
 [1, 0, 1]
RMSE: 0.3325418228923262
R2: -91.07463179681086
explained_variance_score: -75.91872135903957
BS:200

Model predictions:
 [0.39581475655237836, 0.34935203194618225, 0.2397226244211197]

True phenotypes:
 [1, 0, 1]
RMSE: 0.355036112813966
R2: -81.90470016868397
explained_variance_score: -55.169045185682826
BS:200

Model predictions:
 [0.5032438635826111, 0.6120107968648275, 0.2146938592195511]

True phenotypes:
 [1, 0, 1]
RMSE: 0.412676536431645
R2: -13.68297290989031
explained_variance_score: -11.90805589291468
Mean MSE: 0.36675149071264573
BS:200

Model predictions:
 [0.42304914196332294, 0.5106603304545084, 0.36440641681353253]

True phenotypes:
 [1, 0, 1]
RMSE: 0.3325418228923262
R2: -91.07463179681086
explained_variance_score: -75.91872135903957
BS:200

Model predictions:
 [0.39581475655237836, 0.34935203194618225, 0.2397226244211197]

True phenotypes:
 [1, 0, 1]
R

In [43]:
stats_dict_reg = dict()

for batch_size, values in batch_size_dict.items():

    kf = StratifiedKFold(n_splits=3, random_state=rand_seed)
    model_container = []
    X, Y = values[0], values[2]
    X_test, X_train, X_valid, y_test, y_train, y_valid = split_test_valid_train(X=X,
        y=Y,
        test_perc=test_perc,
        train_perc=train_perc,
        valid_perc=0.5, seed=rand_seed)
    X = np.array([*X_train, *X_valid])
    y = np.array([*y_train, *y_valid])
    for train_idx, valid_idx, in kf.split(X=X, y=y):
        i = 1
        outdir_pheno_reg = f'ms_pheno_reg_{batch_size}_{i}'
        i = i +1
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]
        model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                                 nsubset=int(batch_size*len(X_train)/2),
                                 nfilters=[3, 15, 25, 35], coeff_l1=0,
                                 max_epochs=100, nrun=30, learning_rate=None,
                                 ncell=batch_size,
                                 per_sample=True, regression=True,
                                 outdir=outdir_pheno_reg, verbose=False)
        model_container.append(model)
        results = model.results

    mses = []
    for model in model_container:
        print('BS:' + str(batch_size))
        results = model.results
        test_pred = model.predict(X_test)
        print_regression_model_stats(test_pred, y_test)
        mse = mean_squared_error(y_test, test_pred)
        print(f'MSE : {mse}')
        mses.append(mse)
        print(f'For: {batch_size}:')
    print(f'Mean MSE: {str(float(sum(mses) / len(mses)))}')
    stats_dict_reg[batch_size] = {'mean_mse': mse}

print('DONE')

# todo meine samples sind gerade abhängig von CD4, das sollte NICHT so sein

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


27/27 [==============================] - 0s 2ms/step - loss: 0.1907
BS:40

Model predictions:
 [0.3975597620010376, 0.5379279951254526, 0.384190171957016, 0.5513476133346558, 0.21399378776550293, 0.49298224846522015, 0.3262913227081299]

True phenotypes:
 [0, 1, 0, 1, 0, 0, 0]
RMSE: 0.15939231715980692
R2: -11.493063446399747
explained_variance_score: -10.185015315240934
MSE : 0.15939231715980692
For: 40:
BS:40

Model predictions:
 [0.3889720141887665, 0.6375779708226522, 0.43673235177993774, 0.4961541493733724, 0.39168309171994525, 0.520780622959137, 0.5046434005101522]

True phenotypes:
 [0, 1, 0, 1, 0, 0, 0]
RMSE: 0.20093397237249278
R2: -30.062365655733018
explained_variance_score: -24.084240243770903
MSE : 0.20093397237249278
For: 40:
BS:40

Model predictions:
 [0.41417185465494794, 0.8839597702026367, 0.5116775433222452, 0.47358909249305725, 0.3009268045425415

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


28/28 [==============================] - 0s 5ms/step - loss: 0.1189
BS:80

Model predictions:
 [0.8342917362848917, 0.14910167207320532, 0.45938289165496826, 0.6156279444694519, 0.6383257706960043]

True phenotypes:
 [1, 0, 1, 0, 1]
RMSE: 0.17035268185363342
R2: -2.259793738175615
explained_variance_score: -2.189395790781922
MSE : 0.17035268185363342
For: 80:
BS:80

Model predictions:
 [0.8480329314867655, 0.36554428935050964, 0.5821284850438436, 0.6239220499992371, 0.6851538817087809]

True phenotypes:
 [1, 0, 1, 0, 1]
RMSE: 0.16394800461586545
R2: -5.695208589620406
explained_variance_score: -5.677274128528786
MSE : 0.16394800461586545
For: 80:
BS:80

Model predictions:
 [0.8902785380681356, 0.09516919404268265, 0.9400432109832764, 0.9342462221781412, 1.166499376296997]

True phenotypes:
 [1, 0, 1, 0, 

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


...

23/23 [==============================] - 0s 3ms/step - loss: 0.2277
BS:150

Model predictions:
 [0.5752330621083578, 0.4521605273087819, 0.8713473280270895]

True phenotypes:
 [0, 1, 1]
RMSE: 0.21585755786230634
R2: -5.974444097400253
explained_variance_score: -5.937633928915517
MSE : 0.21585755786230634
For: 150:
BS:150

Model predictions:
 [0.5227590699990591, 0.5344700515270233, 0.6882615884145101]

True phenotypes:
 [0, 1, 1]
RMSE: 0.19572533848312593
R2: -33.418243900485734
explained_variance_score: -32.15261487965869
MSE : 0.19572533848312593
For: 150:
BS:150

Model predictions:
 [0.5169672767321268, 0.624157726764679, 0.5162976781527201]

True phenotypes:
 [0, 1, 1]
RMSE: 0.21416017190765824
R2: -82.35231625132101
explained_variance_score: -77.27711033626638
MSE : 0.21416017190765824
For: 150:
Mean MSE: 0


/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


12/12 [==============================] - 0s 11ms/step - loss: 0.3362
BS:200

Model predictions:
 [0.3754891057809194, 0.5112095177173615, 0.5445660253365835]

True phenotypes:
 [1, 0, 0]
RMSE: 0.3159670613180059
R2: -58.09646260533888
explained_variance_score: -54.23132566513723
MSE : 0.3159670613180059
For: 200:
BS:200

Model predictions:
 [0.46477533380190533, 0.5514942606290182, 0.5396826763947805]

True phenotypes:
 [1, 0, 0]
RMSE: 0.2939562513380809
R2: -198.3555102944264
explained_variance_score: -175.0650510880084
MSE : 0.2939562513380809
For: 200:
BS:200

Model predictions:
 [0.4147348503271739, 0.4991930425167084, 0.4962313175201416]

True phenotypes:
 [1, 0, 0]
RMSE: 0.2793248365354731
R2: -181.38469301249103
explained_variance_score: -169.17959530601942
MSE : 0.2793248365354731
For: 200:
Mean M

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


8/8 [==============================] - 0s 3ms/step - loss: 0.1991
BS:250

Model predictions:
 [0.5893989602724711, 0.6272934675216675]

True phenotypes:
 [0, 0]
RMSE: 0.3704441143828136
R2: -1030.8822976175734
explained_variance_score: 0.0
MSE : 0.3704441143828136
For: 250:
BS:250

Model predictions:
 [0.9492155512173971, 0.47430981198946637]

True phenotypes:
 [0, 0]
RMSE: 0.5629899802112149
R2: -8.984948306336928
explained_variance_score: 0.0
MSE : 0.5629899802112149
For: 250:
BS:250

Model predictions:
 [0.9364844560623169, 0.6277322769165039]

True phenotypes:
 [0, 0]
RMSE: 0.6355254739645559
R2: -25.666922056695498
explained_variance_score: 0.0
MSE : 0.6355254739645559
For: 250:
Mean MSE: 0
DONE


In [44]:
stats_dict_reg


KeyboardInterrupt



In [20]:
batch_sizes = [80, 150,200, 250, 300]
batch_size_dict_cd4 = dict()
cluster = 1
for batch_size in batch_sizes:
    ### desease states 1 = RRMS and 0 = NINDC
    selection_pool_rrms_cd4, too_few_data_rrms_cd4 = get_chunks_from_df(rrms_patients2df,
                           freq_df=rrms_patients_freq,
                           desease_state=1,
                           cluster=cluster,
                           batch_size=batch_size)
    selection_pool_nindc_cd4, too_few_data_nindc_cd4 = get_chunks_from_df(nindc_patients2df,
                           freq_df=nindc_patients_freq,
                           desease_state=0,
                           cluster=cluster,
                           batch_size=batch_size)
    #todo make sure list are equally long:
    if len(selection_pool_rrms_cd4) > len(selection_pool_nindc_cd4):
        selection_pool_rrms_cd4 = selection_pool_rrms_cd4[:len(selection_pool_nindc_cd4)]
    elif len(selection_pool_rrms_cd4) < len(selection_pool_nindc_cd4):
        selection_pool_nindc_cd4 = selection_pool_nindc_cd4[:len(selection_pool_rrms_cd4)]

    all_chunks = selection_pool_rrms_cd4 + selection_pool_nindc_cd4
    np.random.shuffle(all_chunks)

    X = [selection[0] for selection in all_chunks]
    cd4 = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]
    batch_size_dict_cd4[batch_size] = (X, cd4, Y)

print('prep done')

prep done


In [22]:
### CD4 Freq regression task
model_container_cd4 = []
cluster=1
stats_dict_reg_cd4 = dict()
for batch_size, values in batch_size_dict_cd4.items():
    # for regression task stratified is wrong since there are no classes
    kf = KFold(n_splits=3, random_state=rand_seed, shuffle=True)
    model_container = []
    freq_idx =[key for key, value in cluster_to_celltype_dict.items() if key==cluster]
    X, cd4 = values[0], values[1]
    cd4 = [series[freq_idx[0]] for series in cd4]
    X_test, X_train, X_valid, cd4_test, cd4_train, cd4_valid = split_test_valid_train(
        X=X,
        y=cd4,
        test_perc=test_perc,
        train_perc=train_perc,
        valid_perc=0.5, seed=rand_seed)
    X = np.array([*X_train, *X_valid])
    cd4 = np.array([*cd4_train, *cd4_valid])
    for train_idx, valid_idx, in kf.split(X=X):
        i = 1
        outdir_pheno_reg_cd4 = f'ms_pheno_reg_cd4_{batch_size}_{i}'
        i = i +1
        X_train, X_valid = X[train_idx], X[valid_idx]
        cd4_train, cd4_valid = cd4[train_idx], cd4[valid_idx]
        model = get_fitted_model(X_train, X_valid, cd4_train, cd4_valid,
                                 nsubset=int(batch_size*len(X_train)/2),
                                 nfilters=[3, 15, 25, 35], coeff_l1=0,
                                 max_epochs=100, nrun=30, learning_rate=None,
                                 ncell=batch_size,
                                 per_sample=True, regression=True,
                                 outdir=outdir_pheno_reg_cd4, verbose=False)
        model_container_cd4.append(model)

    stats_file = open(f"{outdir}_{batch_size}/stats_file.txt", "w+")
    stats_file.write(f"Batchsize: {batch_size}")
    mses = []
    for model in model_container_cd4:
        test_pred = model.predict(X_test)
        train_pred = model.predict(X_train)
        valid_pred = model.predict(X_valid)
        #print_regression_model_stats(test_pred, cd4_test)
        mse_test = mean_squared_error(cd4_test, test_pred)
        mse_train = mean_squared_error(cd4_train, train_pred)
        mse_valid = mean_squared_error(cd4_valid, valid_pred)
        mses.append(mse)
        stats_file.write(f'MSE test {mse_test}')
        stats_file.write(f'MSE train {mse_train}')
        stats_file.write(f'MSE valid {mse_valid}')
        stats_file.write('\n')
    mean_mse = float(sum(mses) / len(mses))
    stats_file.write(f"Mean MSE: {str(mean_mse)}")
    stats_dict_reg_cd4[batch_size] = {'mean_mse': mean_mse}
    stats_file.close()
print('DONE')

2022-02-26 13:23:55.046217: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


28/28 [==============================] - 0s 5ms/step - loss: 0.0143


FileNotFoundError: [Errno 2] No such file or directory: 'out_ms_default_80/stats_file.txt'

In [ ]:
stats_dict_reg_cd4


In [42]:
batch_sizes = [10, 20, 40, 50, 60, 80]
batch_size_dict_b = dict()
cluster = 0
for batch_size in batch_sizes:
    ### desease states 1 = RRMS and 0 = NINDC
    selection_pool_rrms_cd4, too_few_data_rrms_cd4 = get_chunks_from_df(rrms_patients2df,
                           freq_df=rrms_patients_freq,
                           desease_state=1,
                           cluster=cluster,
                           batch_size=batch_size)
    selection_pool_nindc_cd4, too_few_data_nindc_cd4 = get_chunks_from_df(nindc_patients2df,
                           freq_df=nindc_patients_freq,
                           desease_state=0,
                           cluster=cluster,
                           batch_size=batch_size)
    #todo make sure list are equally long:
    if len(selection_pool_rrms_cd4) > len(selection_pool_nindc_cd4):
        selection_pool_rrms_cd4 = selection_pool_rrms_cd4[:len(selection_pool_nindc_cd4)]
    elif len(selection_pool_rrms_cd4) < len(selection_pool_nindc_cd4):
        selection_pool_nindc_cd4 = selection_pool_nindc_cd4[:len(selection_pool_rrms_cd4)]

    all_chunks = selection_pool_rrms_cd4 + selection_pool_nindc_cd4
    np.random.shuffle(all_chunks)

    X = [selection[0] for selection in all_chunks]
    cd4 = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]
    batch_size_dict_b[batch_size] = (X, cd4, Y)

print('prep done')

prep done


In [43]:
### CD4 Freq regression task
model_container_b = []
stats_dict_reg_b = dict()
cluster=0

In [44]:
for batch_size, values in batch_size_dict_b.items():
    # for regression task stratified is wrong since there are no classes
    kf = KFold(n_splits=3, random_state=rand_seed, shuffle=True)
    model_container = []
    freq_idx =[key for key, value in cluster_to_celltype_dict.items() if key==cluster]
    X, b = values[0], values[1]
    b = [series[freq_idx[0]] for series in b]
    X_test, X_train, X_valid, b_test, b_train, b_valid = split_test_valid_train(
        X=X,
        y=b,
        test_perc=test_perc,
        train_perc=train_perc,
        valid_perc=0.5, seed=rand_seed)
    X = np.array([*X_train, *X_valid])
    b = np.array([*b_train, *b_valid])
    i = 1
    for train_idx, valid_idx, in kf.split(X=X):
        outdir_pheno_reg_b = f'ms_pheno_reg_b_{batch_size}_{i}'
        i = i +1
        X_train, X_valid = X[train_idx], X[valid_idx]
        b_train, b_valid = b[train_idx], b[valid_idx]
        model = get_fitted_model(X_train, X_valid, b_train, b_valid,
                                 nsubset=int(batch_size*len(X_train)/2),
                                 nfilters=[3, 15, 25, 35], coeff_l1=0,
                                 max_epochs=100, nrun=30, learning_rate=None,
                                 ncell=batch_size,
                                 per_sample=True, regression=True,
                                 outdir=outdir_pheno_reg_b, verbose=False)
        model_container_b.append(model)

1/1 [==============================] - 0s 18ms/step - loss: 9.8406e-05


In [39]:
stats_file = open(f"{outdir}_{batch_size}/stats_file.txt", "a+")
stats_file.write(f"Batchsize: {batch_size}")
mses = []
for model in model_container_b:
    test_pred = model.predict(X_test)
    train_pred = model.predict(X_train)
    valid_pred = model.predict(X_valid)
    #print_regression_model_stats(test_pred, b_test)
    mse_test = mean_squared_error(b_test, test_pred)
    mse_train = mean_squared_error(b_train, train_pred)
    mse_valid = mean_squared_error(b_valid, valid_pred)
    mses.append(mse)
    stats_file.write(f'MSE test {mse_test}')
    stats_file.write(f'MSE train {mse_train}')
    stats_file.write(f'MSE valid {mse_valid}')
    stats_file.write('\n')
mean_mse = float(sum(mses) / len(mses))
stats_file.write(f"Mean MSE: {str(mean_mse)}")
stats_dict_reg_b[batch_size] = {'mean_mse': mean_mse}
stats_file.close()
print('DONE')

FileNotFoundError: [Errno 2] No such file or directory: 'out_ms_default_80/stats_file.txt'

In [45]:
stats_dict_reg_b

{}

In [46]:
batch_sizes = [10, 20, 40, 50, 60, 80, 100]
batch_size_dict_cd8 = dict()
cluster = 8
for batch_size in batch_sizes:
    ### desease states 1 = RRMS and 0 = NINDC
    selection_pool_rrms_cd8, too_few_data_rrms_cd8 = get_chunks_from_df(rrms_patients2df,
                           freq_df=rrms_patients_freq,
                           desease_state=1,
                           cluster=cluster,
                           batch_size=batch_size)
    selection_pool_nindc_cd8, too_few_data_nindc_cd8 = get_chunks_from_df(nindc_patients2df,
                           freq_df=nindc_patients_freq,
                           desease_state=0,
                           cluster=cluster,
                           batch_size=batch_size)
    #todo make sure list are equally long:
    if len(selection_pool_rrms_cd8) > len(selection_pool_nindc_cd8):
        selection_pool_rrms_cd8 = selection_pool_rrms_cd8[:len(selection_pool_nindc_cd8)]
    elif len(selection_pool_rrms_cd8) < len(selection_pool_nindc_cd8):
        selection_pool_nindc_cd8 = selection_pool_nindc_cd8[:len(selection_pool_rrms_cd8)]

    all_chunks = selection_pool_rrms_cd8 + selection_pool_nindc_cd8
    np.random.shuffle(all_chunks)

    X = [selection[0] for selection in all_chunks]
    cd8 = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]
    batch_size_dict_cd8[batch_size] = (X, cd8, Y)

print('prep done')

prep done


In [47]:
### CD4 Freq regression task
model_container_cd8 = []
stats_dict_reg_cd8 = dict()
outdir_pheno_reg_cd8 =
cluster=8

In [52]:
for batch_size, values in batch_size_dict_cd8.items():
    # for regression task stratified is wrong since there are no classes
    kf = KFold(n_splits=3, random_state=rand_seed, shuffle=True)
    model_container = []
    freq_idx =3
    X, cd8 = values[0], values[1]
    cd8 = [series[freq_idx] for series in cd8]
    X_test, X_train, X_valid, cd8_test, cd8_train, cd8_valid = split_test_valid_train(
        X=X,
        y=cd8,
        test_perc=test_perc,
        train_perc=train_perc,
        valid_perc=0.5, seed=rand_seed)
    X = np.array([*X_train, *X_valid])
    cd8 = np.array([*cd8_train, *cd8_valid])
    i = 1
    for train_idx, valid_idx, in kf.split(X=X):
        outdir_pheno_reg_cd8 = f'ms_pheno_reg_cd8_{batch_size}_{i}'
        i = i +1
        X_train, X_valid = X[train_idx], X[valid_idx]
        cd8_train, cd8_valid = cd8[train_idx], cd8[valid_idx]
        model = get_fitted_model(X_train, X_valid, cd8_train, cd8_valid,
                                 nsubset=int(batch_size*len(X_train)/2),
                                 nfilters=[3, 15, 25, 35], coeff_l1=0,
                                 max_epochs=100, nrun=30, learning_rate=None,
                                 ncell=batch_size,
                                 per_sample=True, regression=True,
                                 outdir=outdir_pheno_reg_cd8, verbose=False)
        model_container_cd8.append(model)

6/6 [==============================] - 0s 7ms/step - loss: 0.0068


In [53]:
model_container_cd8

In [63]:
outdir_cd8 = 'cd8'
stats_file = open(f"{outdir_cd8}_stats_file.txt", "x+")
stats_file.write(f"Batchsize: {batch_size}")
mses = []
for model in model_container_cd8:
    test_pred = model.predict(X_test)
    train_pred = model.predict(X_train)
    valid_pred = model.predict(X_valid)
    #print_regression_model_stats(test_pred, b_test)
    mse_test = mean_squared_error(cd8_test, test_pred)
    mse_train = mean_squared_error(cd8_train, train_pred)
    mse_valid = mean_squared_error(cd8_valid, valid_pred)
    mses.append(mse_test)
    stats_file.write(f'MSE test {mse_test}')
    stats_file.write(f'MSE train {mse_train}')
    stats_file.write(f'MSE valid {mse_valid}')
    stats_file.write('\n')
mean_mse = float(sum(mses) / len(mses))
stats_file.write(f"Mean MSE: {str(mean_mse)}")
stats_dict_reg_cd8[batch_size] = {'mean_mse': mean_mse}
stats_file.close()
print('DONE')

/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature n

DONE


In [64]:
batch_sizes = [1,2,3,4,5,7,10]
batch_size_dict_nkt = dict()
cluster = 3
for batch_size in batch_sizes:
    ### desease states 1 = RRMS and 0 = NINDC
    selection_pool_rrms_nkt, too_few_data_rrms_nkt = get_chunks_from_df(rrms_patients2df,
                           freq_df=rrms_patients_freq,
                           desease_state=1,
                           cluster=cluster,
                           batch_size=batch_size)
    selection_pool_nindc_nkt, too_few_data_nindc_nkt = get_chunks_from_df(nindc_patients2df,
                           freq_df=nindc_patients_freq,
                           desease_state=0,
                           cluster=cluster,
                           batch_size=batch_size)
    #todo make sure list are equally long:
    if len(selection_pool_rrms_nkt) > len(selection_pool_nindc_nkt):
        selection_pool_rrms_nkt = selection_pool_rrms_nkt[:len(selection_pool_nindc_nkt)]
    elif len(selection_pool_rrms_nkt) < len(selection_pool_nindc_nkt):
        selection_pool_nindc_nkt = selection_pool_nindc_nkt[:len(selection_pool_rrms_nkt)]

    all_chunks = selection_pool_rrms_nkt + selection_pool_nindc_nkt
    np.random.shuffle(all_chunks)

    X = [selection[0] for selection in all_chunks]
    nkt = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]
    batch_size_dict_nkt[batch_size] = (X, nkt, Y)

print('prep done')

prep done


In [65]:
##### NKT trial reg
model_container_nkt = []
stats_dict_reg_nkt = dict()
cluster=3

In [66]:
for batch_size, values in batch_size_dict_nkt.items():
    # for regression task stratified is wrong since there are no classes
    kf = KFold(n_splits=3, random_state=rand_seed, shuffle=True)
    model_container = []
    freq_idx =2
    X, nkt = values[0], values[1]
    nkt = [series[freq_idx] for series in nkt]
    X_test, X_train, X_valid, nkt_test, nkt_train, nkt_valid = split_test_valid_train(
        X=X,
        y=nkt,
        test_perc=test_perc,
        train_perc=train_perc,
        valid_perc=0.5, seed=rand_seed)
    X = np.array([*X_train, *X_valid])
    nkt = np.array([*nkt_train, *nkt_valid])
    i = 1
    for train_idx, valid_idx, in kf.split(X=X):
        outdir_pheno_reg_nkt = f'ms_pheno_reg_nkt_{batch_size}_{i}'
        i = i +1
        X_train, X_valid = X[train_idx], X[valid_idx]
        nkt_train, nkt_valid = nkt[train_idx], nkt[valid_idx]
        model = get_fitted_model(X_train, X_valid, nkt_train, nkt_valid,
                                 nsubset=int(batch_size*len(X_train)/2),
                                 nfilters=[3, 15, 25, 35], coeff_l1=0,
                                 max_epochs=100, nrun=30, learning_rate=None,
                                 ncell=batch_size,
                                 per_sample=True, regression=True,
                                 outdir=outdir_pheno_reg_nkt, verbose=False)
        model_container_nkt.append(model)

1/1 [==============================] - 0s 15ms/step - loss: 0.0012


In [67]:
outdir_nkt = 'nkt'
stats_file = open(f"{outdir_nkt}_stats_file.txt", "x+")
stats_file.write(f"Batchsize: {batch_size}")
mses = []
for model in model_container_nkt:
    test_pred = model.predict(X_test)
    train_pred = model.predict(X_train)
    valid_pred = model.predict(X_valid)
    #print_regression_model_stats(test_pred, b_test)
    mse_test = mean_squared_error(nkt_test, test_pred)
    mse_train = mean_squared_error(nkt_train, train_pred)
    mse_valid = mean_squared_error(nkt_valid, valid_pred)
    mses.append(mse_test)
    stats_file.write(f'MSE test {mse_test}')
    stats_file.write(f'MSE train {mse_train}')
    stats_file.write(f'MSE valid {mse_valid}')
    stats_file.write('\n')
mean_mse = float(sum(mses) / len(mses))
stats_file.write(f"Mean MSE: {str(mean_mse)}")
stats_dict_reg_nkt[batch_size] = {'mean_mse': mean_mse}
stats_file.close()
print('DONE')

/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature n

DONE


In [93]:

batch_sizes = [1,2,3,4,5,7,10]
batch_size_dict_nkt = dict()
clusters = [1, 8, 0]




# mean of clusters -> take minimum
min = get_mean_from_clusters(clusters, cluster_to_celltype_dict, patients_clusters_conf_table)
min

36.37096774193548

In [ ]:
for batch_size in batch_sizes:
    ### desease states 1 = RRMS and 0 = NINDC

    selection_pool_rrms_nkt, too_few_data_rrms_nkt = get_chunks_from_df(rrms_patients2df,
                           freq_df=rrms_patients_freq,
                           desease_state=1,
                           cluster=clusters,
                           batch_size=batch_size)
    selection_pool_nindc_nkt, too_few_data_nindc_nkt = get_chunks_from_df(nindc_patients2df,
                           freq_df=nindc_patients_freq,
                           desease_state=0,
                           cluster=cluster,
                           batch_size=batch_size)
    #todo make sure list are equally long:
    if len(selection_pool_rrms_nkt) > len(selection_pool_nindc_nkt):
        selection_pool_rrms_nkt = selection_pool_rrms_nkt[:len(selection_pool_nindc_nkt)]
    elif len(selection_pool_rrms_nkt) < len(selection_pool_nindc_nkt):
        selection_pool_nindc_nkt = selection_pool_nindc_nkt[:len(selection_pool_rrms_nkt)]

    all_chunks = selection_pool_rrms_nkt + selection_pool_nindc_nkt
    np.random.shuffle(all_chunks)

    X = [selection[0] for selection in all_chunks]
    nkt = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]
    batch_size_dict_nkt[batch_size] = (X, nkt, Y)

print('prep done')